In [12]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch import nn
import torch.optim as optimzer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

CATEGORIES = ["buildings", "forest",
              "glacier", "mountain", 
              "sea", "street"]
EPOCHS = 50
TEST_SIZE = 0.30

In [13]:
def load_Intel_Image(dataDirectory : str):
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDirectory, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            final_image = cv2.imread(img_path)
            
            if final_image is None:
                raise Exception("Problem with the images")
            img = cv2.resize(final_image, (64,64)) #Making the Picture 64 x 64
            images.append(img)
            labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))

            
        
            

In [14]:
images, labels = load_Intel_Image("seg_train/")

X_train, X_test, Y_train, Y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE, random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_test: {X_test.shape}")
print(f"This is the shape of Y_Test: {Y_test.shape}")

This is the shape of X_train: (9823, 64, 64, 3)
This is the shape of Y_train: (9823,)
This is the shape of X_test: (4211, 64, 64, 3)
This is the shape of Y_Test: (4211,)


In [15]:
#1) Normalize Input Pixels
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0


#2) We need to Transpose the Matrix because TF (M, H, W, NC) -> Pytorch (M, NC, H, W)
X_train = np.transpose(X_train, (0,3,1,2))
X_test = np.transpose(X_test, (0,3,1,2))

#3) Codificate the Y to Labels Encoder
Label_Encoder = LabelEncoder()
Y_train_encoded = Label_Encoder.fit_transform(Y_train)
Y_test_encoded = Label_Encoder.fit_transform(Y_test)

#4) Convert tensor of Pytorch(The numpy array of TensorFlow)
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train_encoded, dtype=torch.long)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test_encoded, dtype=torch.long)

In [16]:
train_Dataset = TensorDataset(x_train, y_train)
test_DataSet = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_Dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_DataSet, batch_size=64, shuffle=True)


In [ ]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes=6):
        super(CNN_Model, self).__init__()
        
        #First Block
        self.zero_pad = nn.ZeroPad2d(3) #Padding (3,3s)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        #Second Block
        self.covn2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #Third Block
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1,padding=1)
        
        self.fc = nn.Linear(32 * 2 * 2, num_classes)
        